In [1]:
#!/usr/bin/env python
# coding: utf-8

#upload libraries
import pandas as pd
import numpy as np
import json
import sqlite3
from os import listdir
from os.path import isfile, join
from itertools import islice
import re
import progressbar
import csv

In [93]:
# In[17]:
#seperate tweets into their users
def seperate(tweets):
    '''
    takes as input tweets and seperate them into their own users 
    return a dictionary with keys as user ids and values as description and all tweets of this user in input tweets
    '''
    users = dict()
    bar = progressbar.ProgressBar()
    for twt in bar(tweets):
        id_ = twt['user_id']
        try:
            users[id_]['tweets'].append(twt['text'])
        except:
            u = dict()
            #this user does not exist
            u['description'] = twt['text']
            u['tweets'] = [twt['text']]
            users[id_] = u
    return(users)

In [94]:
#keywords for pro your topic
hash_pro = list(pd.read_csv("hashtags_pro.txt", sep = ",", skiprows= list(range(9)), names= ['hashtag'], encoding='utf-8').hashtag)
#hash_pro = [''.join(re.findall('[0-9A-Za-z]', hash_pro[i])) for i in range(len(hash_pro))]
keywords_pro = '|'.join(hash_pro)

#keywords for anti your topic
hash_anti = list(pd.read_csv("hashtags_anti.txt", sep = ",", skiprows= list(range(9)), names= ['hashtag'], encoding='utf-8').hashtag)
#hash_anti = [''.join(re.findall('[0-9A-Za-z]', hash_anti[i])) for i in range(len(hash_anti))]
keywords_anti = '|'.join(hash_anti) #for regex 


def pro_or_anti(description):
    if bool(re.search(keywords_pro, description, re.IGNORECASE)):
        return(0)
    elif bool(re.search(keywords_anti, description, re.IGNORECASE)):
        return(1)
    else:
        return(None)

def pro_anti(users):
    #list of pro, anti tweets
    pro = []
    anti = []
    bar = progressbar.ProgressBar()
    for usr in bar(users.keys()):
        description = str(users[usr]['description'])
        #print(description)
        #see if pro/anti/None
        ind = pro_or_anti(description)
        if ind == 1: #antiocrat
            anti.extend(list(users[usr]['tweets']))
        elif ind == 0: #proublican
            pro.extend(list(users[usr]['tweets']))
        else:
            pass
    return(pro, anti)


In [95]:
keywords_pro

'FreeHongKong |HKPoliceGangsCollusion |HKGestapo |SaveHongKong |twosystems |cansavehk |5DemandsNot1Less |5demandsnotoneless |LiberateHongKong |AntiExtraditionBill |boycottMulan |ShameHKgov |ShameHKpolice |FollowBackHongKong |China70years |PoliceBrutality |hongkongpolicebrutality|antichinazi|shout4HK |humanitariancrisis |followbackhongkong |freedomHK |fightforfreedom |china70yearsofshame |GloryToHongKong|願榮光歸香港 |光復香港 |時代革命 |沙田 |香港人加油 |五大訴求缺一不可|光復香港時代革命  |解散警隊刻不容緩|必須成立獨立調查委員會|齊上齊落 |時代革命 |願榮光歸香港 |十一國殤 |賀佢老母  |十一國殤日 |香港人加油 |光復香港時代革命|沒有暴徒只有暴政|香港加油  '

In [96]:
keywords_anti

'SupportChina |hkmobs |SupportHongKongPolice |hongkongterroristsshouldbejailed |stophkriots |RestorePeaceAndOrderHK |SupportHKSAR |SupportMulan |supportliuyifei |hongkongriot |HKriots |saynotohkviolence |selfloathinghkchinese |stophkpropaganda |onechinapolicy|一个中国 |守护香港 |阿sir我们挺你 |香港是中国的香港 |香港危機 |香港暴徒 |香港废青 |香港警察 |反送中 |反送中條例遊行 |废青你们只是政客的牺牲品 |中国一点都不能少 |严禁插手中国内政 |坚决维护祖国统一  |付国豪 |香港警察 |废青不是中国人|废青必死 |汉奸一个都没有好下场 |香港加油 |香港警察 |我支持香港警察你们可以打我了 |帝国主义休想得逞|支持香港警察 |废青滚出香港 |废青滚出中国 |废青 |香港机场|帝吧'

In [98]:
#create csv where we will store data

with open('Data/labeled_tweets.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    #wr.writerow(['index', 'tweet', 'label'])

In [99]:
DB = r"C:\Users\ericluo04\Documents\GitHub\Bots-Project\Code\1. HK Dataset\HK English\HK English.db"

def get_all_users( json_str = False ):
    conn = sqlite3.connect( DB )
    conn.row_factory = sqlite3.Row # This enables column access by name: row['column_name'] 
    db = conn.cursor()

    rows = db.execute('''
    SELECT * from tweet
    ''').fetchall()

    conn.commit()
    conn.close()

    if json_str:
        return [dict(ix) for ix in rows] #CREATE JSON

    return rows

twtdata = get_all_users(json_str = True)

In [102]:
def get_data(twtdata, n_max, party = None):
    count = 0
    n_tweets = 0
    n = 5000  # Or whatever chunk size you want
    pro, anti = [], []

    #for tweets in iter(lambda: list(islice((json.loads(json.dumps(users))), n)), []):   
    for i in range(0, len(twtdata), n):
        tweets = json.loads(json.dumps(twtdata[i:i+n]))
        users = seperate(tweets)
        pro_, anti_ = pro_anti(users) #this gives us a list of pro and anti tweets
        pro.extend(pro_)
        anti.extend(anti_)
        count += 1
        #save every 10 chunks of data (i.e. 50000 tweets)
        if count % 10 == 0:
            pro = [[i, 0] for i in pro_]
            anti = [[i, 1] for i in anti_]
            if party == 'pro':
                all_ = pro
            elif party == 'anti':
                all_ = anti
            else:
                all_ = pro + anti
            df = pd.DataFrame(all_, columns = ['tweet', 'pro/anti'])
            n_tweets += df.shape[0]
            print(n_tweets)
            #print(df.head(10))
            with open('Data/labeled_tweets.csv', 'a',  encoding = 'utf-8') as f_data:
                df.to_csv(f_data, header=True)
        if n_tweets > n_max:  
            print('stop: maximum number of tweets reached')
            break 

In [104]:
n_max = 100000000 #max_number of tweets
get_data(twtdata, n_max)

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

83


100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

219


100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

...

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

3834


100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

3849


100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################